# Goals:
## -compare change of houing prices with change in imigrants
## -find out wether imigrants tend to buy newly built properties or existing properties


### Importing libraries


In [84]:
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Markdown
import geopandas as gpd
import plotly.express as px
import numpy as np
import seaborn as sns
from shapely.ops import transform
import pyproj
import plotly.graph_objects as go
import geojson

### Setting up the table

In [116]:
table=pd.read_csv("../Data/housing_prices.csv")
table=table.drop(columns=["DATAFLOW","LAST UPDATE","freq"])
states=table["geo"].unique()
#print(table.groupby(by=["geo","OBS_VALUE"]).head())
countries_coordinates_json=gpd.read_file("../Data/geography/geography_processed.geojson")
#print(countries_cordinates_json.loc[:,["Country","geometry"]])
#print(countries_coordinates_json.columns)
countries_coordinates={countries_coordinates_json.loc[:,"Country"][i]:countries_coordinates_json.loc[:,"geometry"][i] 
                       for i in range(len(countries_coordinates_json)) 
                       if countries_coordinates_json.loc[:,"Country"][i] in states }
#print(table.query("purchase=='Total' & geo=='Slovakia' & 'change' not in unit").loc[:,"OBS_VALUE"].sort_values())

def create_country_query(state,year="",type='Total'):
    print(table.query(f"purchase=='{type}' & geo=='{state}' & unit.str.startswith('A')").loc[:,"OBS_VALUE"].sort_values())
    print(state)

for i in states:
    create_country_query(i,type="Purchases of existing dwellings")

4866    -6.8
4868    -5.3
4867    -4.8
4865    -4.1
4869    -1.3
4864    -0.4
4870    -0.2
4871     1.1
4829     2.6
4827     2.8
4830     3.0
4836     3.7
4821     3.9
4824     4.1
4835     4.5
4828     4.5
4826     4.6
4831     4.8
4843     4.9
4819     5.3
4817     5.4
4818     5.4
4863     5.5
4848     5.5
4833     5.5
4816     5.5
4844     5.6
4837     5.9
4846     6.0
4849     6.0
4834     6.0
4822     6.0
4845     6.1
4823     6.2
4841     6.2
4850     6.2
4842     6.2
4838     6.2
4853     6.3
4851     6.3
4840     6.9
4832     7.2
4820     7.5
4847     7.5
4855     7.7
4839     7.9
4852     8.0
4825     8.6
4854     8.7
4856    10.6
4858    12.6
4862    12.8
4861    13.5
4857    13.8
4860    15.1
4859    15.4
Name: OBS_VALUE, dtype: float64
Austria
4885    -1.2
4906    -0.6
4886    -0.4
4905    -0.3
4904    -0.2
        ... 
4875     9.7
4876    10.1
4874    11.5
4873    12.7
4872    12.8
Name: OBS_VALUE, Length: 76, dtype: float64
Belgium
4962   -28.0
4963   -26.3
4961   -21.